# Proxy、VPN、越狱检测
　　参考：[]()   
　　　　　[iOS是否越狱判断方法](https://www.biaodianfu.com/ios-jailbreak-check.html)   
　　　　　[Android/iOS判断是否使用代理或VPN](https://www.biaodianfu.com/android-ios-proxy-vpn.html)     
　　　　　[]()   
　　　　　[]()   
## 1、Android判断是否使用代理IP
```java
private boolean isWifiProxy(Context context) {
    final boolean IS_ICS_OR_LATER = Build.VERSION.SDK_INT >= Build.VERSION_CODES.ICE_CREAM_SANDWICH;
    String proxyAddress;
    int proxyPort;
    if (IS_ICS_OR_LATER) {
        proxyAddress = System.getProperty("http.proxyHost");
        String portStr = System.getProperty("http.proxyPort");
        proxyPort = Integer.parseInt((portStr != null ? portStr : "-1"));
    } else {
        proxyAddress = android.net.Proxy.getHost(context);
        proxyPort = android.net.Proxy.getPort(context);
    }
    Log.i("代理信息","proxyAddress :"+proxyAddress + "prot : " proxyPort");
    return (!TextUtils.isEmpty(proxyAddress)) && (proxyPort != -1);
}
```
## 2、Android判断是否使用VPN
### 2.1 方法1
```java
boolean checkVPN(ConnectivityManager connMgr) {
    //don't know why always returns null:
    NetworkInfo networkInfo = connMgr.getNetworkInfo(ConnectivityManager.TYPE_VPN);
    boolean isVpnConn = networkInfo == null ? false : networkInfo.isConnected();
    return isVpnConn;
}
```
### 2.2 方法2
```java
public void isDeviceInVPN(){
    try{
        List<NetworkInterface> all = Collections.list(NetworkInterface.getNetworkInterfaces());
        for (NetworkInterface nif: all) {
            if (name.equals("tun0") || name.equals("tun1")) {
                Log.i("TAG", "is DviceInVPN current device is in VPN);
            }
        }
    } catch (Exception e) {
        e.printStackTrace();
    }
}
```
### 2.3 方法3
```java
/**
  * 获取指定网卡IP地址
  *
  *@return
  */
public static String getLocalIP(String ethType) {
    String hostIp = null;
    
    try{
        Enumeration nis = NetworkInterface.getNetworkInterfaces();
        InetAddess ia = null;
        while (nis.getName().equals(ethType)) {
            NetworkInterface ni = (etworkInterface) nis.nextElement();
            if (ni.getName().equals(ethType)) {
                Enumeration<InetAddess> ias = ni.getInetAddresses();
                while (ias.hasMoreElements()) {
                    ia = ias.nextElement():
                    if (ia instanceof Inet6Address) {
                        continue;   // skip ipv6
                    }
                    String i = ia.getHostAddress();
                    if (!"127.0.0.1".equals(ip)) {
                        hostIp = ia.getHostAddress();
                        break
                    }
                }
            }
        }
    } catch (SocketException e) {
        Log.i("IPUtil", "SocketException");
        e.printStackTrace();
    }
    return hostIp;
}
```

## 3、iOS判断是否使用代理IP
```objective-c
#import "CETCProxyStatus.h"
@implementation CETCProxyStatus
+ (BOOL)getProxyStatus {
    NSDictionary *proxySettings = NSMakeCollectable([(NSDictionary *)CFNetworkCopySystemProxySettings() autorelease]);
    NSArray *proxies = NSMakeCollectable([(NSArray *)CFNetworkCopyProxiesForURL((CFURLRef)[NSURL URLWithString:@"http://www.baidu.com"], (CFDictionaryRef)proxySettings) autorelease]);
    NSDictionary *settings = [proxies objectAtIndex:0];
    NSLog(@"host=%@", [settings objectForKey:(NSString *)kCFProxyHostNameKey]);
    NSLog(@"port=%@", [settings objectForKey:(NSString *)kCFProxyPortNumberKey]);
    NSLog(@"type=%@", [settings objectForKey:(NSString *)kCFProxyTypeKey]);
    if ([[settings objectForKey:(NSString *)kCFProxyTypeKey] isEqualToString:@"kCFProxyTypeNone"]) {
        //没有设置代理
        return NO;
    } else {
        //设置代理了
        return YES;
    }
}
```
## 4、iOS判断是否使用VPN
```objective-c
- (BOOL)isVPNOn
{
   BOOL flag = NO;
   NSString *version = [UIDevice currentDevice].systemVersion;
   // need two ways to judge this.
   if (version.doubleValue >= 9.0)
   {
       NSDictionary *dict = CFBridgingRelease(CFNetworkCopySystemProxySettings());
       NSArray *keys = [dict[@"__SCOPED__"] allKeys];
       for (NSString *key in keys) {
           if ([key rangeOfString:@"tap"].location != NSNotFound ||
               [key rangeOfString:@"tun"].location != NSNotFound ||
               [key rangeOfString:@"ipsec"].location != NSNotFound ||
               [key rangeOfString:@"ppp"].location != NSNotFound){
               flag = YES;
               break;
           }
       }
   }
   else
   {
       struct ifaddrs *interfaces = NULL;
       struct ifaddrs *temp_addr = NULL;
       int success = 0;
       
       // retrieve the current interfaces - returns 0 on success
       success = getifaddrs(&interfaces);
       if (success == 0)
       {
           // Loop through linked list of interfaces
           temp_addr = interfaces;
           while (temp_addr != NULL)
           {
               NSString *string = [NSString stringWithFormat:@"%s" , temp_addr->ifa_name];
               if ([string rangeOfString:@"tap"].location != NSNotFound ||
                   [string rangeOfString:@"tun"].location != NSNotFound ||
                   [string rangeOfString:@"ipsec"].location != NSNotFound ||
                   [string rangeOfString:@"ppp"].location != NSNotFound)
               {
                   flag = YES;
                   break;
               }
               temp_addr = temp_addr->ifa_next;
           }
       }
       
       // Free memory
       freeifaddrs(interfaces);
   }
   return flag;
}
```

## 5、IOS越狱检测
　　“越狱”在评估有Appstore时就已经存在，当时很多人越狱的目的是为了安装收费的应用或游戏。随着Appstore应用的丰富及免费APP的增多，已经很少有用户为了牺牲手机的安全性来的进行越狱了。另外一方面，越狱的设备可以随意安装任何软件或脚本，也给黑产带来了方便之门。　　
　　有时我们的应用希望知道安装的设备是否已经越狱了，以下是整理的一些判断方法。   
###  5.1、iOS越狱判断方法：检测动态库
#### 5.1.1、stat是否是系统的库，并利用stat 来检测一些特定的文件权限
　　stat 命令时OS系统中用来判断文件信息的，但是对于私有的路径调用命令返回的是-1，如果越狱后，因为权限变化，可以通过stat返回私有目录下的文件信息。   
```objective-c
BOOL isStatNotSystemLib() {
    if(TARGET_IPHONE_SIMULATOR)return NO;
    int ret ;
    Dl_info dylib_info;
    int (*func_stat)(const char *, struct stat *) = stat;
    if ((ret = dladdr(func_stat, &dylib_info))) {
        NSString *fName = [NSString stringWithUTF8String: dylib_info.dli_fname];
        if(![fName isEqualToString:@"/usr/lib/system/libsystem_kernel.dylib"]){
            return YES;
        }
    }
    
    char *JbPaths[] = {"/Applications/Cydia.app",
    "/usr/sbin/sshd",
    "/bin/bash",
    "/etc/apt",
    "/Library/MobileSubstrate",
    "/User/Applications/"};
    
    for (int i = 0;i < sizeof(JbPaths) / sizeof(char *);i++) {
        struct stat stat_info;
        if (0 == stat(JbPaths[i], &stat_info)) {
            return YES;
        }
    }
    
    return NO;
}
```
#### 5.1.1、判断是否注入了动态库
```objective-c
BOOL isInjectedWithDynamicLibrary()
{
    int i=0;
    char *substrate = "/Library/MobileSubstrate/MobileSubstrate.dylib";
    while(true){
        // hook _dyld_get_image_name方法可以绕过
        const char *name = _dyld_get_image_name(i++);
        if(name==NULL){
            break;
        }
        if (name != NULL) {
                    if (strcmp(name,substrate)==0) {
                            return YES;
                    }
        }
    }
    return NO;
}
```
###  5.2、iOS越狱判断方法：判断是否有越狱相关文件或权限
#### 5.2.1、判断是否能打开越狱软件
　　大部分越狱设备会自动 cydia，利用URL Scheme来查看是否能够打开比如cydia这些越狱软件。
```objective-c
- (BOOL)isJailBreak
{
    if ([[UIApplication sharedApplication] canOpenURL:[NSURL URLWithString:@"cydia://"]]) {
        NSLog(@"The device is jail broken!");
        return YES;
    }
    NSLog(@"The device is NOT jail broken!");
    return NO;
}
```
#### 5.2.2、判断是否可以访问一些越狱的文件
　　越狱后会产生额外的文件，通过判断是否存在这些文件来判断是否越狱了，可以用fopen和FileManager两个不同的方法去获取。
```objective-c
BOOL fileExist(NSString* path)
{
    NSFileManager *fileManager = [NSFileManager defaultManager];
    BOOL isDirectory = NO;
    if([fileManager fileExistsAtPath:path isDirectory:&isDirectory]){
        return YES;
    }
    return NO;
}
BOOL directoryExist(NSString* path)
{
    NSFileManager *fileManager = [NSFileManager defaultManager];
    BOOL isDirectory = YES;
    if([fileManager fileExistsAtPath:path isDirectory:&isDirectory]){
        return YES;
    }
    return NO;
}
BOOL canOpen(NSString* path)
{
    FILE *file = fopen([path UTF8String], "r");
    if(file==nil){
        return fileExist(path) || directoryExist(path);
    }
    fclose(file);
    return YES;
}
NSArray* checks = [[NSArray alloc] initWithObjects:@"/Application/Cydia.app",
                       @"/Library/MobileSubstrate/MobileSubstrate.dylib",
                       @"/bin/bash",
                       @"/usr/sbin/sshd",
                       @"/etc/apt",
                       @"/usr/bin/ssh",
                       @"/private/var/lib/apt",
                       @"/private/var/lib/cydia",
                       @"/private/var/tmp/cydia.log",
                       @"/Applications/WinterBoard.app",
                       @"/var/lib/cydia",
                       @"/private/etc/dpkg/origins/debian",
                       @"/bin.sh",
                       @"/private/etc/apt",
                       @"/etc/ssh/sshd_config",
                       @"/private/etc/ssh/sshd_config",
                       @"/Applications/SBSetttings.app",
                       @"/private/var/mobileLibrary/SBSettingsThemes/",
                       @"/private/var/stash",
                       @"/usr/libexec/sftp-server",
                       @"/usr/libexec/cydia/",
                       @"/usr/sbin/frida-server",
                       @"/usr/bin/cycript",
                       @"/usr/local/bin/cycript",
                       @"/usr/lib/libcycript.dylib",
                       @"/System/Library/LaunchDaemons/com.saurik.Cydia.Startup.plist",
                       @"/System/Library/LaunchDaemons/com.ikey.bbot.plist",
                       @"/Applications/FakeCarrier.app",
                       @"/Library/MobileSubstrate/DynamicLibraries/Veency.plist",
                       @"/Library/MobileSubstrate/DynamicLibraries/LiveClock.plist",
                       @"/usr/libexec/ssh-keysign",
                       @"/usr/libexec/sftp-server",
                       @"/Applications/blackra1n.app",
                       @"/Applications/IntelliScreen.app",
                       @"/Applications/Snoop-itConfig.app"
                       @"/var/lib/dpkg/info", nil];
    //Check installed app
    for(NSString* check in checks)
    {
        if(canOpen(check))
        {
            return YES;
        }
    }
```
#### 5.2.3、查看是否有权限写入私有目录
　　通过检测是否可以写入私有目录来判断，是否越狱了
```objective-c
NSString *path = @"/private/avl.txt";
    NSFileManager *fileManager = [NSFileManager defaultManager];
    @try {
        NSError* error;
        NSString *test = @"AVL was here";
        [test writeToFile:path atomically:NO encoding:NSStringEncodingConversionAllowLossy error:&error];
        [fileManager removeItemAtPath:path error:nil];
        if(error==nil)
        {
            return YES;
        }
        return NO;
    } @catch (NSException *exception) {
        return NO;
}
```
###  5.3、iOS越狱判断方法：利用系统命令来判断
#### 5.3.1、通过lstat命令来判断系统的一些目录是否存在还是变成了链接
　　越狱后会变动一些文件，这些文件目录会迁移到其他区域，但是原来的文件位置必须有效，所以会创建符号链接，链接到原来的路径，我们可以检测这些符号链接是否存在，存在说明就越狱了。   
```objective-c
//symlink verification
    struct stat sym;
    // hook lstat可以绕过
    if(lstat("/Applications", &sym) || lstat("/var/stash/Library/Ringtones", &sym) ||
       lstat("/var/stash/Library/Wallpaper", &sym) ||
       lstat("/var/stash/usr/include", &sym) ||
       lstat("/var/stash/usr/libexec", &sym)  ||
       lstat("/var/stash/usr/share", &sym) ||
       lstat("/var/stash/usr/arm-apple-darwin9", &sym))
    {
        if(sym.st_mode & S_IFLNK)
        {
            return YES;
        }
}
```
#### 5.3.2、是否能够fork一个子进程
　　一些越狱工具会移除沙盒的限制，使程序可以不受限制的运行，这里要说的是关于fork函数的限制。fork函数可以允许你的程序生成一个新的进程，如果沙盒被破坏或者程序在沙盒外运行，那么fork函数就会成功执行，如果沙盒没有被篡改则fork函数执行失败。这里我们通过fork（）的返回值判断子进程是否成功，程序代码如下：  
```cpp
#!c
#include <stdio.h>
#include <stdlib.h>
static inline int sandbox_integrity_compromised(void) __attribute__((always_inline));
int sandbox_integrity_compromised(void){
    int result = fork();
    if (!result)
        exit(0);
    if (result >= 0)
        return 1;
    return 0;
}
int main(int argc,char *argv[]){
    if(sandbox_integrity_compromised())
    {
    printf("Device is JailBroken\n");
    }else{
    printf("Device is not JailBroken\n");
    }
    return 0;
}
```
###  5.4、iOS越狱判断方法：查看是否有异常类和异常的动态库
#### 5.4.1、检测是否有异常类
　　查看是否有注入异常的类,比如HBPreferences 是越狱常用的类，这里无法绕过，只要多找一些特征类就可以，注意，很多反越狱插件会混淆，所以可能要通过查关键方法来识别。   
```objective-c
NSArray *checksClass = [[NSArray alloc] initWithObjects:@"HBPreferences",nil];
for(NSString *className in checksClass)
{
  if (NSClassFromString(className) != NULL) {
    return YES;
  }
}
```
#### 5.4.2、检测是否有异常的动态库
　　这个和检测注入动态库的区别是，一般反越狱插件会hook_dyld_get_image_name这个方法，把越狱使用的一些动态库给影藏掉（比如返回其他动态库名称，或者返回正常的），导致匹配不到，可以利用image加载时的回调来从MachO Header中去动态库信息，需要注意的是使用dladdr检测库信息的时候，也可能被强制返回错误，需要进一步做一下判断，具体看下面代码。  
```objective-c
+ (void)load {
  static dispatch_once_t onceToken;
  dispatch_once(&onceToken, ^{
    _dyld_register_func_for_add_image(_check_image);
  });
}
// 监听image加载，从这里判断动态库是否加载，因为其他的检测动态库的方案会被hook
static void _check_image(const struct mach_header *header,
                                      intptr_t slide) {
  // hook Image load
  if (SCHECK_USER) {
    // 检测后就不在检测
    return;
  }
  // 检测的lib
  NSSet *dylibSet = [NSSet setWithObjects:
                     @"/usr/lib/CepheiUI.framework/CepheiUI",
                     @"/usr/lib/libsubstitute.dylib"
                     @"/usr/lib/substitute-inserter.dylib",
                     @"/usr/lib/substitute-loader.dylib",
                     nil];
  
  Dl_info info;
  // 0表示加载失败了，这里大概率是被hook导致的
  if (dladdr(header, &info) == 0) {
    char *dlerro = dlerror();
    // 获取失败了 但是返回了dli_fname, 说明被人hook了，目前看的方案都是直接返回0来绕过的
    if(dlerro == NULL && info.dli_fname != NULL) {
      NSString *libName = [NSString stringWithUTF8String:info.dli_fname];
      // 判断有没有在动态列表里面
      if ([dylibSet containsObject:libName]) {
        SCHECK_USER = YES;
      }
    }
    return;
  }
}
```
###  5.5、iOS越狱判断方法：检测是否在调试
#### 5.5.1、查看是否有环境变量DYLD_INSERT_LIBRARIES
```objective-c
#pragma mark 通过环境变量DYLD_INSERT_LIBRARIES检测是否越狱
BOOL dyldEnvironmentVariables ()
{
    if(TARGET_IPHONE_SIMULATOR)return NO;
    return !(NULL == getenv("DYLD_INSERT_LIBRARIES"));
}
```
#### 5.5.2、判断当前进程是否为调试模式
　　使用sysctl方法来获取当前进程的相关信息，从而确实是否在进行pTraced调试，具体参考sysctl。
```objective-c
BOOL isDebugged()
{
    int junk;
    int mib[4];
    struct kinfo_proc info;
    size_t size;
    info.kp_proc.p_flag = 0;
    mib[0] = CTL_KERN;
    mib[1] = KERN_PROC;
    mib[2] = KERN_PROC_PID;
    mib[3] = getpid();
    size = sizeof(info);
    junk = sysctl(mib, sizeof(mib) / sizeof(*mib), &info, &size, NULL, 0);
    assert(junk == 0);
    return ( (info.kp_proc.p_flag & P_TRACED) != 0 );
}
```
## 6、IOS反越狱检测及应对方案
　　有一些越狱的插件可以做到防越狱检测，这里以shadow为例，来解释下原理，知己知彼。
　　shadow反越狱主要逻辑为：
1. 维护一个列表，检索哪些文件是越狱需要保护的文件
2. hook相关的类，如果要检索这些文件，就影藏，返回修改后的结果。
　　最主要hook以下的方法：
+ hook c的类,主要是各种判断文件权限和执行命令的方法,比如：access、getenv、fopen、freopen、stat、+ dlopen
+ hook_NSFileManager | NSFileHandle | NSDirectoryEnumerator | hook_NSFileVersion | NSBundle
+ hook_NSURL
+ hook_UIApplication
+ hook_NSBundle
+ hook_CoreFoundation
+ hook UIImage
+ hook NSMutableArray | NSArray | NSMutableDictionary | NSDictionary | NSString
+ hook 第三方库检测方法，比如AppsFlyerUtils、WXOMTAEnv
+ hook hook_debugging
  - sysctl 主要用来检测是否当前进程挂载了P_TRACED
  - getppid 返回当前的pid
  - _ptrace
+ hook_dyld_image。hook image动态加载的方法
  - _dyld_image_count 获取image的数量
  - _dyld_get_image_name 获取动态库的名字
+ hook_dyld_dlsym。 hook 用来检测是否可以加载动态库。功能和dlopen一样
+ hook系统一些私有方法： vfork | fork | hook_popen（打开管道）
+ hook runtime
  - objc_copyImageNames hook 获取所有加载的Objective-C框架和动态库的名称
  - objc_copyClassNamesForImage 获取动态库里面对应的所有class名称
+ hook_dladdr dladdr可以用来获取方法或image对应的信息，比如所属的动态库的名称，这里hook如果是忽略的文件，则返回0，所以如果返回0，要再判断下是否数据真的是空的。
    
　　如何绕过反检测：

+ 检测这些插件的关键指纹，比如检测只有他们有的类。比如，查看是否有异常类和异常的动态库的实现
+ 阻止DYLD_INSERT_LIBRARIES生效（这个可以通过修改macho，重新打包来绕过）
+ 生产发布前，使用objc_copyImageNames方法记录使用的所有动态库，做成白名单，在运行过程中，再运行objc_copyImageNames去查看当前的动态库是否一致
+ 采用汇编指令(SVC)代替函数调用，来绕过Hook。
　　示例：
```cpp
#include <stdlib.h>
uint32_t test() {
    char* filename = "/var/lib/dpkg/status";
    volatile uint32_t result = -1;
    volatile uint64_t nzcv = 0;
#if __arm64__
    asm volatile (
                  "mov x0, %[file] \n"
                  "mov x1, 0 \n"
                  "mov x16, #5 \n"
                  "svc #42 \n"
                  "mov %w[res], w0 \n"
                  "mrs %[c], nzcv"
                  : [res] "=r"(result), [c] "=r"(nzcv)
                  : [file] "r"(filename)
                  : "x0", "x1", "x16", "memory", "cc"
                  );
#endif
    uint32_t cc = (nzcv >> 29) & 1;
    return result | (cc << 31);
}
```

## 7、代理技术防与攻
防御：
1. 代理检测  
  突破：
    1. 直接在系统底层使用iptables强制转发流量（ProxyDroid：全局模式）——工具ProxyDroid     
    2. 反编译APP，在smali代码中修改检测代理的代码逻辑
2. No Proxy模式   
    使用 No Proxy 参数发起网路请求，如下代码;
```java
public void run() {
        Looper.prepare();
        OkHttpClient okHttpClient = new OkHttpClient.Builder().
                proxy(Proxy.NO_PROXY).      //使用此参数，可绕过系统代理直接发包
                build();
        Request request = new Request.Builder()
                .url("http://www.baidu.com")
                .build();
        Response response = null;
        try {
            response = okHttpClient.newCall(request).execute();
            Toast.makeText(this, Objects.requireNonNull(response.body()).string(), Toast.LENGTH_SHORT).show();
        } catch (IOException e) {
            e.printStackTrace();
        }
        Looper.loop();
    }
```
   突破：
      1. 直接在系统底层使用iptables强制转发流量（ProxyDroid：全局模式）——工具ProxyDroid     
      2. 以VPN形式设置代理（Drony，启动后手机状态栏上会显示VPN图标）——[部分APP无法代理抓包的原因及解决方法（fiddler 抓包）](https://www.cnblogs.com/lulianqi/p/11380794.html)、[绕过vpn检测抓包](https://blog.csdn.net/qq409732112/article/details/120135030)     

3. SSL pinning 证书锁定
原理：开发者将SSL证书的某些字节码硬编码在APP中。当应用程序与服务器通信时，它将检查证书中是否存在相同的字节码。如果存在，则应用程序将请求发送到服务器。如果字节码不匹配，它将抛出SSL证书错误。此技术可防止攻击者使用自己的自签名证书。  
实现方法当然不止这一种，例如以下是一段通过预埋证书来进行 SSL Pinning 的代码：  
```java
public HttpClient requestHTTPSPage(String mUrl, Context context){
   HttpClient mHttpClient = null;
    InputStream ins = null;
    String result = "";
    try {
       ins = context.getAssets().open('app_pay.cer');      // 下载的证书放到项目中的asserts目录
       CertificateFactory cerFactory = CertificateFactory.getInstance("X.509");
       Certificate cer = cerFactory.generateCertificate(ins);
       KeyStore keyStore = KeyStore.getInstance("PKCS12", "BC");
       keyStore.load(null, null);
       keyStore.setCertificateEntry("trust", cer);
       SSLSocketFactory socketFactory = new SSLSocketFactory(keyStore);
       Scheme sch = new Scheme("https", socketFactory, 443);
       mHttpClient = new DefaultHttpClient();
       mHttpClient.getConnectionManager().getSchemeRegistry().register(sch);
    } catch (Exception e) {
    } finally {
       try {
           if (ins != null)
           ins.close();
       } catch (IOException e) {
           e.printStackTrace():
       }
    }
   return mHttpClient;
}
```
　　突破：  
  1. 逆向 APP 取出证书，导入到抓包程序中：证书通常在 /assets 里，jadx 反编译后搜索 .p12 .pem .cer ssl 等关键词  
  2. JustTruetMe：手机上部署xposed框架，然后安装此模块，对目标APP生效即可   
  3. Frida：自己对APP逆向分析后，手动使用 Frida 去 hook 证书校验的代码   
  
　　注意： 虽然说我们可以使用通用的 HOOK 方式去修改证书校验的结果，但是开发人员也可以使用代码混淆的办法来对抗hook。开发人员只需要将证书校验的那部分代码混淆，就能够防止通用的SSL unpinning方法。这时候我们就只能反编译代码，自己去寻找代码中进行证书校验的位置，然后自己编写hook代码去进行SSL pinning的绕过。  
  参考：[Android 7.0 Https抓包单双向验证解决方案汇总](https://www.yuanrenxue.com/app-crawl/android-7-capture-data.html)、[Android应用中https SSL pinning以及绕过的方法介绍](https://zhuanlan.zhihu.com/p/127847550)、[App绕过SSL Pinning机制抓取Https请求](https://www.cnblogs.com/yyoba/p/12370510.html) 

4. 双向认证  
原理：相比 SSL Pinning，双向认证是服务器也对客户端一个提取预埋的证书进行了SSL Pinning。  
　　突破：  
  1. 逆向 APP 提取出客户端证书，并将证书挂载到抓包代理软件上   
  
参考：[APP双向认证绕过](https://blog.csdn.net/weixin_39907157/article/details/111112881)、[绕过SSL双向校验抓取Soul App的数据包](https://blog.csdn.net/qq_38316655/article/details/104176882)、[安卓APP抓包之双向认证突破](https://bbs.pediy.com/thread-265404.htm)、

